In [1]:
import pdfplumber
import os
import re
import pandas as pd
import sys
import datetime
import calendar
from tqdm import tqdm

In [2]:
dirPath = '.\\announcements\\muyuan\\monthly'
fileList = os.listdir(dirPath)

In [3]:
def get_data(patten, text):
    try:
        data = float(re.findall(patten, text)[0])
        return data
    except:
        return 'NA'

In [4]:
hogData = []
for file in tqdm(fileList):
    filePath = dirPath+'\\'+file
    with pdfplumber.open(filePath) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()
        text = ''.join(text.split()) #去除空白符
        text = text.replace(',','')

        p_year = '一、(.*?)年'
        p_month = '一、.*?年(.*?)月份'
        p_year = '一、(.*?)年'
        p_month = '一、.*?年(.*?)月份'
        year = re.findall(p_year, text)[0]
        month = re.findall(p_month, text)[0]
        if month == '1-2':
            startDate = datetime.datetime(int(year), 1, 1)
            endDate = datetime.datetime(int(year), 2, calendar.monthrange(int(year), 2)[1])
        else:
            startDate = datetime.datetime(int(year), int(month), 1)
            endDate = datetime.datetime(int(year), int(month), calendar.monthrange(int(year), int(month))[1])
        qDate = str((startDate.month-1)//3+1)+'Q'+str(startDate.year%100)
        
        p_total = '销售生猪(.*?)万头' 
        p_hog = '商品猪(.*?)万头.*?均价'
        p_piglet = '仔猪(.*?)万头.*?均价'
        p_sow = '种猪(.*?)万头.*?均价'
        p_toSubs = '子公司.*?销售生猪(.*?)万头'
        p_hogPrice = '均价(.*?)元'
        p_hogRev = '销售收入(.*?)亿元'
        p_total_qtr = '均价.*?共销售生猪(.*?)万头'
        p_hog_qtr = '均价.*?其中商品猪(.*?)万头'
        p_piglet_qtr = '均价.*?仔猪(.*?)万头'
        p_sow_qtr = '均价.*?种猪(.*?)万头'
        p_breedingSowInv = '能繁母猪存栏为(.*?)万头'
        

        total = get_data(p_total, text)
        hog = get_data(p_hog, text)
        piglet = get_data(p_piglet, text.replace('销售',''))
        sow = get_data(p_sow, text)
        toSubs = get_data(p_toSubs, text)
        hogPrice = get_data(p_hogPrice, text)
        hogRev = get_data(p_hogRev, text)
        totalQtr = get_data(p_total_qtr, text)
        hogQtr = get_data(p_hog_qtr, text)
        pigletQtr = get_data(p_piglet_qtr, text)
        sowQtr = get_data(p_sow_qtr, text)
        breedingSowInv = get_data(p_breedingSowInv, text)

        hogData.append([startDate, endDate, qDate, total, hog, piglet, sow, toSubs, hogPrice, hogRev, totalQtr, hogQtr, pigletQtr, sowQtr, breedingSowInv])
        sys.stdout.flush()

100%|██████████| 100/100 [01:15<00:00,  1.33it/s]


In [5]:
tag = ['start date', 'end date', 'qdate','total', 'hog', 'piglet', 'sow', 'to subsidiaries', 'hog price', 'hog revenue', 'total qtr', 'hog qtr', 'piglet qtr', 'sow qtr', 'breeding sow inventory']
hogDf = pd.DataFrame(hogData, columns=tag)
hogDf.set_index('start date', inplace = True)
hogDf.sort_index(inplace=True)

In [6]:
outputPath = 'muyuan_monthly.csv'
hogDf.to_csv(outputPath,sep=',',index=True,header=True) 